In [1]:
cd ..

/home/abhishekabhishek/git/AQT-KimGroup


In [14]:
from keys import keys
from qiskit import IBMQ, QuantumRegister, QuantumCircuit, quantum_info

%matplotlib inline

In [6]:
# Setup account
provider = IBMQ.enable_account(keys['ibmq_token'])

In [7]:
IBMQ.save_account(keys['ibmq_token'])

In [8]:
devices = provider.backends(simulator=True, operational=True)
for x in devices:
    print(x.name(), x.configuration().n_qubits, x.status().pending_jobs)

ibmq_qasm_simulator 32 0
simulator_statevector 32 0
simulator_mps 100 0
simulator_extended_stabilizer 63 0
simulator_stabilizer 5000 0


In [9]:
simulator_backend = provider.backend.ibmq_qasm_simulator

Reproducing Qiskit MLE


The following was part of the old notebook but is now deprecated in qiskit - 
`from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter`

In [39]:
from qiskit_experiments.library import StateTomography, tomography

In [18]:
# No. of qubits
n_qubits = 3

# Initialize the quantum register and circuit
q_reg = QuantumRegister(n_qubits)
ghz_circ = QuantumCircuit(q_reg)

# Add operations to the circuit to prepage a GHZ state
ghz_circ.h(0)
for idx in range(n_qubits-1):
    ghz_circ.cx(idx, idx+1)

ghz_circ.draw()

┌───┐          
q3_0: ┤ H ├──■───────
      └───┘┌─┴─┐     
q3_1: ─────┤ X ├──■──
           └───┘┌─┴─┐
q3_2: ──────────┤ X ├
                └───┘

In [22]:
qst_exp = StateTomography(ghz_circ)
len(qst_exp.circuits())

27

Following the introduction to the experiment framework at `https://qiskit.org/documentation/experiments/apidocs/framework.html#module-qiskit_experiments.framework`

In [41]:
qst_exp_data = qst_exp.run(simulator_backend)

In [42]:
qst_exp_data

ExperimentData(StateTomography, 06a7b4f6-98fa-4fda-9cee-991ddc720952, job_ids=['636c55240565be2b3600742c'], metadata=<3 items>)

In [43]:
qst_exp_data.save()

Experiment cannot be saved because no experiment service is available. An experiment service is available, for example, when using an IBM Quantum backend.


In [44]:
st_analysis = tomography.StateTomographyAnalysis()
qst_exp_res = st_analysis.run(qst_exp_data)

In [45]:
qst_exp_res.data()

[{'counts': {'000': 2005, '111': 1995},
  'job_id': '636c55240565be2b3600742c',
  'metadata': {'clbits': [0, 1, 2], 'm_idx': [0, 0, 0]},
  'shots': 4000,
  'meas_level': <MeasLevel.CLASSIFIED: 2>},
 {'counts': {'000': 1013, '011': 942, '100': 1042, '111': 1003},
  'job_id': '636c55240565be2b3600742c',
  'metadata': {'clbits': [0, 1, 2], 'm_idx': [0, 0, 1]},
  'shots': 4000,
  'meas_level': <MeasLevel.CLASSIFIED: 2>},
 {'counts': {'000': 1019, '011': 1005, '100': 980, '111': 996},
  'job_id': '636c55240565be2b3600742c',
  'metadata': {'clbits': [0, 1, 2], 'm_idx': [0, 0, 2]},
  'shots': 4000,
  'meas_level': <MeasLevel.CLASSIFIED: 2>},
 {'counts': {'000': 951, '010': 990, '101': 1017, '111': 1042},
  'job_id': '636c55240565be2b3600742c',
  'metadata': {'clbits': [0, 1, 2], 'm_idx': [0, 1, 0]},
  'shots': 4000,
  'meas_level': <MeasLevel.CLASSIFIED: 2>},
 {'counts': {'000': 482,
   '001': 490,
   '010': 517,
   '011': 468,
   '100': 511,
   '101': 517,
   '110': 513,
   '111': 502},
  'j

In [46]:
qst_exp_data = qst_exp_res.data()
keys = [qst_exp_data[i].keys() for i in range(len(qst_exp_data))]
print(keys)

[dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'sho

In [49]:
pauli_basis = tomography.basis.PauliMeasurementBasis

rho_li = tomography.fitters.linear_inversion(None, None, None,
                                             measurement_basis=pauli_basis)

TypeError: linear_inversion() missing 3 required positional arguments: 'shot_data', 'measurement_data', and 'preparation_data'

In [51]:
pauli_basis = tomography.basis.PauliMeasurementBasis()
